In [1]:
!pip install ultralytics opencv-python pillow boto3

In [2]:
import cv2

cap = cv2.VideoCapture("flood1.mp4")
ret, frame = cap.read()
cap.release()

cv2.imwrite("frame.jpg", frame)

error: OpenCV(4.13.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:1193: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'


In [ ]:
import boto3
import base64
import json

bedrock = boto3.client("bedrock-runtime")

with open("frame.jpg", "rb") as f:
    img = base64.b64encode(f.read()).decode()

prompt = """
You are analyzing a flood scene image.

Return JSON:
{
  "people_trapped": true/false,
  "vehicles_submerged": true/false,
  "severity": low/medium/high
}
"""

response = bedrock.invoke_model(
    modelId="amazon.nova-pro-v1",
    body=json.dumps({
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image", "source": {"type": "base64", "data": img}}
                ]
            }
        ]
    })
)

nova_output = json.loads(response["body"].read())
print(nova_output)

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
results = model("frame.jpg")
boxes = results[0].boxes

In [ ]:
depth_cm = None

for box in boxes:
    cls = int(box.cls[0])
    label = model.names[cls]

    if label == "car":
        x1,y1,x2,y2 = box.xyxy[0]
        height_px = y2 - y1
        waterline_px = height_px * 0.5
        depth_cm = (waterline_px / height_px) * 70
        break

In [ ]:
output = {
    "water_depth_cm": int(depth_cm) if depth_cm else None,
    "severity": nova_output.get("severity"),
    "people_trapped": nova_output.get("people_trapped"),
    "vehicles_submerged": nova_output.get("vehicles_submerged")
}

print(output)

In [ ]:
import json
with open("flood_result.json", "w") as f:
    json.dump(output, f)